In [1]:
! source ../envsim/bin/activate

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
You should consider upgrading via the '/scratch/users/ocagatan19/envsim/bin/python3 -m pip install --upgrade pip' command.


In [3]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple, https://download.pytorch.org/whl/cu116
     |████████████████████████████████| 24.2 MB 18 kB/s  eta 0:00:01
     |████████████████████████████████| 4.2 MB 6.2 MB/s eta 0:00:01
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

torchvision 0.14.1+cu116 requires torch==1.13.1, but you'll have torch 1.13.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you'll have torch 1.13.0 which is incompatible.
You should consider upgrading via the '/scratch/users/ocagatan19/envsim/bin/python3 -m pip install --upgrade pip' command.


In [4]:
!pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
You should consider upgrading via the '/scratch/users/ocagatan19/envsim/bin/python3 -m pip install --upgrade pip' command.


In [14]:
from transformers import AutoModel,AutoConfig,TrainingArguments
from simcse.model.bert_roberta_models import BertForBarlow

In [26]:
from typing import Optional

In [27]:
confing = AutoConfig.from_pretrained("experiments/unsup-barlow/unsup-barlow-bert-base-uncased/")

In [28]:
import numpy as np

In [29]:
import torch

In [30]:
from dataclasses import dataclass, field
from transformers.file_utils import (
    cached_property,
    torch_required,
    is_torch_available,
    is_torch_tpu_available,
)


In [31]:
@dataclass
class BarlowTrainingArguments(TrainingArguments):



    # Evaluation
    ## By default, we evaluate STS (dev) during training (for selecting best checkpoints) and evaluate
    ## both STS and transfer tasks (dev) at the end of training. Using --eval_transfer will allow evaluating
    ## both STS and transfer tasks (dev) during training.
    eval_transfer: bool = field(
        default=False,
        metadata={"help": "Evaluate transfer task dev sets (in validation)."},
    )

    augmentation_method: str = field(
        default="dropout", metadata={"help": "Augmentation method to create positive pairs for unsupervised training."}
    )

    lambd: float = field(
        default= 0.01, metadata={"help": "Weight for covariance loss"},
    )


    do_proj: bool = field(
        default=False, metadata={"help": "Boolean value to add MLP"}
    )
    batch_size:int = field(
        default=512, metadata = {"help": "Integer value to the "}
    )

    freeze_bert: bool = field(
        default=False, metadata={"help": "Boolean value to freeze BERT"}
    )

    proj_output_dim: str = field(
        default="4096-4096-768", metadata={"help": "Projection output dimension"}
    )


    @cached_property
    @torch_required
    def _setup_devices(self) -> "torch.device":
        logger.info("PyTorch: setting up devices")
        if self.no_cuda:
            device = torch.device("cpu")
            self._n_gpu = 0
        elif is_torch_tpu_available():
            device = xm.xla_device()
            self._n_gpu = 0
        elif self.local_rank == -1:
            # if n_gpu is > 1 we'll use nn.DataParallel.
            # If you only want to use a specific subset of GPUs use `CUDA_VISIBLE_DEVICES=0`
            # Explicitly set CUDA to the first (index 0) CUDA device, otherwise `set_device` will
            # trigger an error that a device index is missing. Index 0 takes into account the
            # GPUs available in the environment, so `CUDA_VISIBLE_DEVICES=1,2` with `cuda:0`
            # will use the first GPU in that env, i.e. GPU#1
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            # Sometimes the line in the postinit has not been run before we end up here, so just checking we're not at
            # the default value.
            self._n_gpu = torch.cuda.device_count()
        else:
            # Here, we'll use torch.distributed.
            # Initializes the distributed backend which will take care of synchronizing nodes/GPUs
            #
            # deepspeed performs its own DDP internally, and requires the program to be started with:
            # deepspeed  ./program.py
            # rather than:
            # python -m torch.distributed.launch --nproc_per_node=2 ./program.py
            if self.deepspeed:
                from .integrations import is_deepspeed_available

                if not is_deepspeed_available():
                    raise ImportError(
                        "--deepspeed requires deepspeed: `pip install deepspeed`."
                    )
                import deepspeed

                deepspeed.init_distributed()
            else:
                torch.distributed.init_process_group(backend="nccl")
            device = torch.device("cuda", self.local_rank)
            self._n_gpu = 1

        if device.type == "cuda":
            torch.cuda.set_device(device)

        return device

In [33]:
from transformers import MODEL_FOR_MASKED_LM_MAPPING

MODEL_CONFIG_CLASSES = list(MODEL_FOR_MASKED_LM_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [34]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """

    # Huggingface's original arguments
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The model checkpoint for weights initialization."
            "Don't set if you want to train a model from scratch."
        },
    )

    hidden_dropout_prob: Optional[float] = field(
        default=0.1,
        metadata={"help":"Dropout probability for generating positive pairs."}
    )

    loss_elements: Optional[str] = field(
        default="cov-inv",
        metadata={"help":"Sub-loss calculations to form total loss."}
    )

    tensorboard_path: Optional[str] = field(
        default="tensorboard",
        metadata={"help":"Path to save tansorboard trackings."}
    )

    tensorboard_save_frequency: Optional[int] = field(
        default=100,
        metadata={"help":"Frequency to save tensorboard info."}
    )

    model_type: Optional[str] = field(
        default=None,
        metadata={
            "help": "If training from scratch, pass a model type from the list: "
            + ", ".join(MODEL_TYPES)
        },
    )
    ssl_type: Optional[str] = field(default="simcse",)
    config_name: Optional[str] = field(
        default=None,
        metadata={
            "help": "Pretrained config name or path if not the same as model_name"
        },
    )
    tokenizer_name: Optional[str] = field(
        default=None,
        metadata={
            "help": "Pretrained tokenizer name or path if not the same as model_name"
        },
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={
            "help": "Where do you want to store the pretrained models downloaded from huggingface.co"
        },
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={
            "help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."
        },
    )
    model_revision: str = field(
        default="main",
        metadata={
            "help": "The specific model version to use (can be a branch name, tag name or commit id)."
        },
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )

    # SimCSE's arguments
    temp: float = field(default=0.05, metadata={"help": "Temperature for softmax."})
    pooler_type: str = field(
        default="cls",
        metadata={
            "help": "What kind of pooler to use (cls, cls_before_pooler, avg, avg_top2, avg_first_last)."
        },
    )
    hard_negative_weight: float = field(
        default=0,
        metadata={
            "help": "The **logit** of weight for hard negatives (only effective if hard negatives are used)."
        },
    )
    do_mlm: bool = field(
        default=False, metadata={"help": "Whether to use MLM auxiliary objective."}
    )
    mlm_weight: float = field(
        default=0.1,
        metadata={
            "help": "Weight for MLM auxiliary objective (only effective if --do_mlm)."
        },
    )
    mlp_only_train: bool = field(
        default=False, metadata={"help": "Use MLP only during training"}
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    # Huggingface's original arguments.
    dataset_name: Optional[str] = field(
        default=None,
        metadata={"help": "The name of the dataset to use (via the datasets library)."},
    )
    dataset_config_name: Optional[str] = field(
        default=None,
        metadata={
            "help": "The configuration name of the dataset to use (via the datasets library)."
        },
    )
    overwrite_cache: bool = field(
        default=False,
        metadata={"help": "Overwrite the cached training and evaluation sets"},
    )
    validation_split_percentage: Optional[int] = field(
        default=5,
        metadata={
            "help": "The percentage of the train set used as validation set in case there's no validation split"
        },
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )

    # SimCSE's arguments
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The training data file (.txt or .csv)."}
    )

    max_seq_length: Optional[int] = field(
        default=32,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated."
        },
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "Whether to pad all samples to `max_seq_length`. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch."
        },
    )
    mlm_probability: float = field(
        default=0.15,
        metadata={
            "help": "Ratio of tokens to mask for MLM (only effective if --do_mlm)"
        },
    )

    def __post_init__(self):
        if (
            self.dataset_name is None
            and self.train_file is None
            and self.validation_file is None
        ):
            raise ValueError(
                "Need either a dataset name or a training/validation file."
            )
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in [
                    "csv",
                    "json",
                    "txt",
                ], "`train_file` should be a csv, a json or a txt file."


In [35]:
training_args = torch.load("experiments/unsup-barlow/unsup-barlow-bert-base-uncased/training_args.bin")

In [36]:
training_args

BarlowTrainingArguments(output_dir='experiments/unsup-barlow-bert-base-uncased', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=256, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, learning_rate=1e-06, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=1000, log_level='passive', log_level_replica='passive', log_on_each_node=True, logging_dir='experiments/unsup-barlow-bert-base-uncased/runs/Dec23_00-47-23_rk02.kuacc.ku.edu.tr', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=500, logging_nan_inf_filter=True, save_strategy=<IntervalStrat

In [37]:
from transformers import HfArgumentParser

In [42]:
parser = HfArgumentParser(
    (ModelArguments, DataTrainingArguments,BarlowTrainingArguments)
)

In [45]:
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")


_StoreAction(option_strings=['-f', '--fff'], dest='fff', nargs=None, const=None, default='1', type=None, choices=None, help='a dummy argument to fool ipython', metavar=None)

In [46]:
 model_args, data_args, training_args = parser.parse_args_into_dataclasses()

usage: ipykernel_launcher.py [-h] [--model_name_or_path MODEL_NAME_OR_PATH] [--hidden_dropout_prob HIDDEN_DROPOUT_PROB]
                             [--loss_elements LOSS_ELEMENTS] [--tensorboard_path TENSORBOARD_PATH]
                             [--tensorboard_save_frequency TENSORBOARD_SAVE_FREQUENCY] [--model_type MODEL_TYPE] [--ssl_type SSL_TYPE]
                             [--config_name CONFIG_NAME] [--tokenizer_name TOKENIZER_NAME] [--cache_dir CACHE_DIR]
                             [--use_fast_tokenizer [USE_FAST_TOKENIZER]] [--no_use_fast_tokenizer] [--model_revision MODEL_REVISION]
                             [--use_auth_token [USE_AUTH_TOKEN]] [--temp TEMP] [--pooler_type POOLER_TYPE]
                             [--hard_negative_weight HARD_NEGATIVE_WEIGHT] [--do_mlm [DO_MLM]] [--mlm_weight MLM_WEIGHT]
                             [--mlp_only_train [MLP_ONLY_TRAIN]] [--dataset_name DATASET_NAME] [--dataset_config_name DATASET_CONFIG_NAME]
                             [-

SystemExit: 2

In [48]:
model = BertForBarlow(config=confing,training_args=training_args,model_args = ModelArguments)

In [53]:
model.load_state_dict(torch.load("experiments/unsup-barlow/unsup-barlow-bert-base-uncased/pytorch_model.bin"))


<All keys matched successfully>